<a href="https://colab.research.google.com/github/Shyam-Khokhariya/ML-Python/blob/master/C16_RNN_Character_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
with open("2265.txt",'r',encoding="utf-8") as f:
    text=f.read()
text=text[16246:]
# print(text)


In [4]:
char=set(text)
char2int={ch:i for i,ch in enumerate(char)}
print(char2int)

int2char=dict(enumerate(char))
text_int = np.array([char2int[ch] for ch in text],dtype=np.int32)
print(int2char)

{'z': 0, '.': 1, 'j': 2, 'e': 3, '1': 4, 'b': 5, 'n': 6, '?': 7, 'v': 8, 'O': 9, '(': 10, 'u': 11, 'y': 12, ',': 13, 'G': 14, 'H': 15, 'L': 16, 'B': 17, 'R': 18, 'A': 19, 'x': 20, 'k': 21, 'h': 22, 'D': 23, 'Q': 24, 'M': 25, 'q': 26, 'P': 27, '-': 28, 'f': 29, 'p': 30, 'C': 31, 'W': 32, 'i': 33, '[': 34, 's': 35, 'I': 36, ';': 37, 'g': 38, 'r': 39, 'l': 40, 'N': 41, 'w': 42, ':': 43, 't': 44, 'c': 45, 'Y': 46, ')': 47, 'E': 48, ']': 49, '\n': 50, 'S': 51, 'd': 52, 'm': 53, 'T': 54, "'": 55, '&': 56, 'V': 57, 'Z': 58, 'K': 59, 'a': 60, ' ': 61, 'F': 62, 'o': 63, '!': 64}
{0: 'z', 1: '.', 2: 'j', 3: 'e', 4: '1', 5: 'b', 6: 'n', 7: '?', 8: 'v', 9: 'O', 10: '(', 11: 'u', 12: 'y', 13: ',', 14: 'G', 15: 'H', 16: 'L', 17: 'B', 18: 'R', 19: 'A', 20: 'x', 21: 'k', 22: 'h', 23: 'D', 24: 'Q', 25: 'M', 26: 'q', 27: 'P', 28: '-', 29: 'f', 30: 'p', 31: 'C', 32: 'W', 33: 'i', 34: '[', 35: 's', 36: 'I', 37: ';', 38: 'g', 39: 'r', 40: 'l', 41: 'N', 42: 'w', 43: ':', 44: 't', 45: 'c', 46: 'Y', 47: ')', 

In [0]:
def reshape_data(sequence,batch_size,num_step):
    tot_batch_length=batch_size*num_step
    num_batch=int(len(sequence)/tot_batch_length)
    if num_batch*tot_batch_length +1 > len(sequence):
        num_batch=num_batch-1

    #truncate sequence to get rid of remaining character that do not make full batch
    x=sequence[:num_batch*tot_batch_length]
    y=sequence[1:num_batch*tot_batch_length+1]
    ##Split x & y into list batches of sequence
    x_batch_split=np.split(x,batch_size)
    y_batch_split=np.split(y,batch_size)
    ##Stack batches together
    ##batch_size X tot_batch_length
    x=np.stack(x_batch_split)
    y=np.stack(y_batch_split)
    return x,y

In [0]:
def create_batch_generator(data_x,data_y,num_step):
    batch_size,tot_batch_length=data_x.shape
    num_batch=int(tot_batch_length/num_step)
    for b in range(num_batch):
        yield(data_x[:,b*num_step:(b+1)*num_step],data_y[:,b*num_step:(b+1)*num_step])


In [0]:
def get_top_char(prob,char_size,top_n=3):
    p=np.squeeze(prob)
    p[np.argsort(p)[:-top_n]]=0.0
    p=p/np.sum(p)
    ch_id=np.random.choice(char_size,1,p=p)[0]
    return ch_id


In [0]:
import tensorflow as tf
import os


In [0]:
class CharRNN(object):
    def __init__(self,num_classes,batch_size=64,
                 num_steps=100,lstm_size=128,
                 num_layers=1,learning_rate=0.001,keep_prob=0.5,
                 grad_clip=5,sampling=False):
        self.num_classes=num_classes
        self.batch_size=batch_size
        self.num_steps=num_steps
        self.lstm_size=lstm_size
        self.num_layers=num_layers
        self.learning_rate=learning_rate
        self.keep_prob=keep_prob
        self.grad_clip=grad_clip

        self.g=tf.Graph()
        with self.g.as_default():
            tf.set_random_seed(123)
            self.build(sampling=sampling)
            self.saver=tf.train.Saver()
            self.init_op=tf.global_variables_initializer()


    def build(self,sampling):
        if sampling==True:
            batch_size,num_steps=1,1
        else:
            batch_size,num_steps=self.batch_size,self.num_steps

        tf_x=tf.placeholder(tf.int32,shape=[batch_size,num_steps],name="tf_x")
        tf_y=tf.placeholder(tf.int32,shape=[batch_size,num_steps],name="tf_y")
        tf_keepprob=tf.placeholder(tf.float32,name="tf_keepprob")

        #One Hot Encoding
        x_onehot=tf.one_hot(tf_x,depth=self.num_classes)
        y_onehot=tf.one_hot(tf_y,depth=self.num_classes)

        #Build Multilayer RNN Cell
        cells=tf.contrib.rnn.MultiRNNCell([
            tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(self.lstm_size),output_keep_prob=tf_keepprob)
            for _ in range(self.num_layers)])

        #Define Initial State
        self.initial_state=cells.zero_state(batch_size,tf.float32)

        #Run each sequence step through RNN
        lstm_output,self.final_state=tf.nn.dynamic_rnn(cells,x_onehot,initial_state=self.initial_state)
        print(" << lstm_output >> ",lstm_output)

        seq_output_reshaped=tf.reshape(lstm_output,shape=[-1,self.lstm_size],name="seq_output_reshaped")

        logits=tf.layers.dense(inputs=seq_output_reshaped,
                               units=self.num_classes,activation=None,name="logits")

        proba=tf.nn.softmax(logits,name="probabilities")
        y_reshaped=tf.reshape(y_onehot,shape=[-1,self.num_classes],name="y_reshaped")

        cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits=logits,labels=y_reshaped),name="cost")

        #Gradient Cliping to avoid "exploding gradients"
        tvars=tf.trainable_variables()
        grad,_=tf.clip_by_global_norm(tf.gradients(cost,tvars),self.grad_clip)

        optimizer=tf.train.AdamOptimizer(self.learning_rate)
        train_op=optimizer.apply_gradients(zip(grad,tvars),name="train_op")

    def train(self,train_x,train_y,num_epochs,ckpt_dir="./model/"):
        #Create checkpoint directory if not present
        if not os.path.exists(ckpt_dir):
            os.mkdir(ckpt_dir)

        with tf.Session(graph=self.g) as sess:
            sess.run(self.init_op)

            n_batches=int(train_x.shape[1]/self.num_steps)
            iterations=n_batches*num_epochs
            for epoch in range(num_epochs):

                #Train Network
                new_state=sess.run(self.initial_state)
                loss=0
                ##Mini Batch Generator
                bgen=create_batch_generator(train_x,train_y,self.num_steps)
                for b,(batch_x,batch_y) in enumerate(bgen,1):
                    itr=epoch*n_batches+b
                    feed={"tf_x:0":batch_x,
                          "tf_y:0":batch_y,
                          "tf_keepprob:0":self.keep_prob,
                          self.initial_state:new_state}
                    batch_cost,_,new_state=sess.run(["cost:0","train_op",self.final_state],feed_dict=feed)
                    if itr%10==0:
                        print("Epoch %d/%d Iteration %d | Training Loss : %.4f"%(epoch+1,num_epochs,itr,batch_cost))

                #Save the trained Model
                self.saver.save(sess,os.path.join(ckpt_dir,"language_modeling.ckpt"))


    def sample(self,output_length,ckpt_dir,starter_seq="The "):
        observed_seq=[c for c in starter_seq]
        with tf.Session(graph=self.g) as sess:
            self.saver.restore(sess,tf.train.latest_checkpoint(ckpt_dir))
            ## 1: run model using starter sequence
            new_state=sess.run(self.initial_state)
            for ch in starter_seq:
                x=np.zeros((1,1))
                x[0,0]=char2int[ch]
                feed={"tf_x:0":x,
                      "tf_keepprob:0":1.0,
                      self.initial_state:new_state}
                prob,new_state=sess.run(["probabilities:0",self.final_state],feed_dict=feed)

            ch_id=get_top_char(prob,len(char))
            observed_seq.append(int2char[ch_id])

            ## 2: run model using updated observed sequence
            for i in range(output_length):
                x[0,0]=ch_id
                feed={"tf_x:0":x,
                      "tf_keepprob:0":1.0,
                      self.initial_state:new_state}
                prob,new_state=sess.run(["probabilities:0",self.final_state],feed_dict=feed)
                ch_id=get_top_char(prob,len(char))
                observed_seq.append(int2char[ch_id])
        return "".join(observed_seq)


In [18]:
batch_size=64
num_step=100
train_x,train_y=reshape_data(text_int,batch_size,num_step)
rnn=CharRNN(num_classes=len(char),batch_size=batch_size)
rnn.train(train_x,train_y,num_epochs=100,ckpt_dir="./model-100/")


 << lstm_output >>  Tensor("rnn/transpose_1:0", shape=(64, 100, 128), dtype=float32)
Epoch 1/100 Iteration 10 | Training Loss : 3.6897
Epoch 1/100 Iteration 20 | Training Loss : 3.3698
Epoch 2/100 Iteration 30 | Training Loss : 3.2959
Epoch 2/100 Iteration 40 | Training Loss : 3.2475
Epoch 2/100 Iteration 50 | Training Loss : 3.2343
Epoch 3/100 Iteration 60 | Training Loss : 3.2116
Epoch 3/100 Iteration 70 | Training Loss : 3.1867
Epoch 4/100 Iteration 80 | Training Loss : 3.1674
Epoch 4/100 Iteration 90 | Training Loss : 3.1481
Epoch 4/100 Iteration 100 | Training Loss : 3.1279
Epoch 5/100 Iteration 110 | Training Loss : 3.1163
Epoch 5/100 Iteration 120 | Training Loss : 3.0688
Epoch 6/100 Iteration 130 | Training Loss : 3.0327
Epoch 6/100 Iteration 140 | Training Loss : 3.0040
Epoch 6/100 Iteration 150 | Training Loss : 2.9581
Epoch 7/100 Iteration 160 | Training Loss : 2.9404
Epoch 7/100 Iteration 170 | Training Loss : 2.8910
Epoch 8/100 Iteration 180 | Training Loss : 2.8399
Epoch 

In [22]:

np.random.seed(123)
rnn2=CharRNN(len(char),sampling=True)
print(rnn2.sample(500,starter_seq="Hello ",ckpt_dir="./model-100/"))

 << lstm_output >>  Tensor("rnn/transpose_1:0", shape=(1, 1, 128), dtype=float32)
INFO:tensorflow:Restoring parameters from ./model-100/language_modeling.ckpt
Hello the marsere the well

   Ham. Why she my Lord, and the partion ase thes ale the with
The should his tane to me in the somes of the Palles,
Wer this to merest and the to the mone,
And andert the she ther alles to the well, to heare,
As he sheld what the wealles to the will,
In hin he hath my tould she the thing the touene
thes it as the weate self an the selfe, this that the Songis,
And weld whe it it and the mantes
  Hor. This she to the mend thes that,
To that the pass ist the mestere a that sou
